In [1]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import make_SIR_graph, make_beds_graph

In [2]:
os.chdir('models')
import simulate_pandemic as simp
from policies import policies, costs

Loading Graph...Done!


In [3]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [4]:
def run_simulation(policy, step_size = 7, days = 280):
    pop_matrix = simp.init_infection(.0001)

    data = []
    actions = []
   
    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break
            
        # Choose a new policy at each week
        if day % step_size == 1:
            action = policy(day)
            restrictions = policies[action]
            actions.append(action)

        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix, key=lambda x: x[0]))[:, 1])

    #return data, actions, contacts_infected, pop_matrix
    return data, actions

In [5]:
def unrestricted(day):
    return 'Unrestricted'

def early_end(day):
    if day < 35:
        return 'Unrestricted'
    if day <= 126: 
        return 'Light Quarantine'
    return 'Social Distancing'
    
def intermitent_rest(day):
    actions = ['Social Distancing', 'Social Distancing', 'Hard Quarantine', 'Hard Quarantine', ]*50
    return actions[int(day/7)]

def random_pol(day):
    return np.random.choice(list(policies.keys()))

def lq(day):
    if day < 35:
        return 'Unrestricted'
    return 'Light Quarantine'

In [6]:
from joblib import Parallel, delayed

sims = Parallel(n_jobs=-1)(delayed(run_simulation)(intermitent_rest) for i in range(100))
results = [pd.DataFrame([pd.Series(d).value_counts() for d in s[0]]).fillna(0) for s in tqdm(sims)]
df_mean = pd.concat([r for r in results]).groupby(level=0).mean()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:41<00:00,  2.39it/s]


In [11]:
make_beds_graph(df_mean, sims[0][1], 7, 'average_curve_inter', max_range=0.0025*1.1, make_df=False)